In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install -q langchain langchain-community langchain-text-splitters chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.6 MB/s eta 0:00:0

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from pathlib import Path
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [ ]:
# qcvn_path = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/raw/text/QUY CHUẨN KỸ THUẬT QUỐC GIA VỀ BÁO HIỆU ĐƯỜNG BỘ.pdf")
# cautao_path = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/raw/text/cấu tạo và sửa chữa.pdf")

# def load_pdf_all_text(path: Path):
#     loader = PyPDFLoader(str(path))
#     docs = loader.load()     # mỗi trang -> 1 Document(text, metadata={page, source,...})
#     full_text = "\n".join(d.page_content for d in docs)
#     return docs, full_text

# qcvn_docs, qcvn_text = load_pdf_all_text(qcvn_path)
# cautao_docs, cautao_text = load_pdf_all_text(cautao_path)

# print("Số trang QCVN:", len(qcvn_docs))
# print("Số trang Cấu tạo:", len(cautao_docs))
# print(qcvn_text[:1000])   # xem thử 1000 ký tự đầu


In [ ]:
# qcvn_text_path = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/outputs/qcvn41_2024_bao_hieu_duong_bo.txt")
# cautao_text_path = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/outputs/cautao_suachua_xeoto.txt")

# qcvn_text_path.write_text(qcvn_text, encoding="utf-8")
# cautao_text_path.write_text(cautao_text, encoding="utf-8")

In [5]:
# Đường dẫn tới 2 file text bạn đã trích
qcvn_txt_path = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/outputs/cautao_suachua_xeoto.txt")
cautao_txt_path = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/outputs/qcvn41_2024_bao_hieu_duong_bo.txt")

def load_txt_as_document(path: Path) -> Document:
    text = path.read_text(encoding="utf-8")
    return Document(
        page_content=text,
        metadata={
            "source": str(path),
            "file_name": path.name,
        }
    )

qcvn_doc = load_txt_as_document(qcvn_txt_path)
cautao_doc = load_txt_as_document(cautao_txt_path)

all_docs = [qcvn_doc, cautao_doc]
print("Độ dài QCVN:", len(qcvn_doc.page_content))
print("Độ dài Cấu tạo:", len(cautao_doc.page_content))


Độ dài QCVN: 36159
Độ dài Cấu tạo: 340628


In [6]:
law_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=True,
    separators=[
        # CHƯƠNG I, Chương II, chương III,...
        r"(?im:^\s*chương\s+[ivxlcdm]+\s.*)",
        # MỤC 1, Mục 2,...
        r"(?im:^\s*mục\s+\d+\.?.*)",
        # ĐIỀU 1, Điều 2,...
        r"(?im:^\s*điều\s+\d+\.?.*)",
        # Khoản 1. 2. 3. ở đầu dòng
        r"\n{1,}\d+\.\s",
        # Đoạn
        r"\n{2,}",
        # Dòng
        r"\n",
        # Khoảng trắng
        r" ",
        # Fallback
        r"",
    ],
)

chunked_docs = law_splitter.split_documents(all_docs)
print("Tổng số chunk:", len(chunked_docs))
print(chunked_docs[0].page_content[:500])
print(chunked_docs[0].metadata)


Tổng số chunk: 477
TỔNG CỤC ĐƯỜNG BỘ VIỆT NAM

GIÁO TRÌNH
CẤU TẠO VÀ SỬA CHỮA THÔNG THƯỜNG XE Ô TÔ
Dùng cho các lớp đào tạo lái xe ôtô

HÀ NỘI, NĂM 2017

CHỦ BIÊN: TỔNG CỤC ĐƯỜNG BỘ VIỆT NAM
BIÊN SOẠN: Ths. VƯƠNG TRỌNG MINH
HIỆU ĐÍNH: KS. NGUYỄN THẮNG QUÂN
KS. TRẦN QUỐC TUẤN
Ths. LƯƠNG DUYÊN THỐNG

LỜI NÓI ĐẦU

Giáo trình khung Cấu tạo và sửa chữa thông thường xe ôtô được biên soạn trên cơ sở chương trình đào tạo lái xe ôtô theo quy định của Bộ Giao thông vận tải. Cấu tạo và sửa chữa thông thường xe ôtô là một tro
{'source': '/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/outputs/cautao_suachua_xeoto.txt', 'file_name': 'cautao_suachua_xeoto.txt'}


In [10]:
txt_path = "/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chunks/chunk_text_law2.txt"

with open(txt_path, "w", encoding="utf-8") as f:
    for i, doc in enumerate(chunked_docs):
        f.write(f"===== CHUNK {i} =====\n")
        f.write(doc.page_content)
        f.write("\n\n\n")

print("Đã lưu file TXT tại:", txt_path)


Đã lưu file TXT tại: /content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chunks/chunk_text_law2.txt


In [9]:
# Model embedding tiếng Việt
embedding_model = HuggingFaceEmbeddings(
    model_name="AITeamVN/Vietnamese_Embedding",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # thường dùng cho search cosine
)


/tmp/ipython-input-1720378661.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# Thư mục chứa ChromaDB
persist_directory = "/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chroma_text_law2"
Path(persist_directory).mkdir(parents=True, exist_ok=True)

vectordb = Chroma.from_documents(
    documents=chunked_docs,
    embedding=embedding_model,
    persist_directory=persist_directory,
)

# Lưu DB xuống đĩa
vectordb.persist()
print("Đã embed và lưu vào Chroma tại:", persist_directory)


Đã embed và lưu vào Chroma tại: /content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chroma_text_law2


/tmp/ipython-input-3642308913.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [ ]:
retriever = vectordb.as_retriever(
    search_type="similarity",      # hoặc "mmr"
    search_kwargs={"k": 5},        # lấy top 5 chunk
)

In [ ]:
# Ví dụ truy vấn
query = "Theo loại nhiên liệu sử dụng ôtô chia thành các loại nào?"

docs = retriever.invoke(query)  # LangChain >= 0.3 dùng .invoke()

print(f"Tìm được {len(docs)} chunk\n")
for i, d in enumerate(docs, start=1):
    print(f"===== KẾT QUẢ {i} =====")
    print("Source:", d.metadata.get("source_file"))
    print("Page:", d.metadata.get("page"))
    print("Nội dung:")
    print(d.page_content[:500], "...")
    print()

Tìm được 5 chunk

===== KẾT QUẢ 1 =====
Source: None
Page: None
Nội dung:
1.2.2 - Theo loại nhiên liệu sử dụng
Theo loại nhiên liệu sử dụng ôtô chia thành các loại:
- Xe ôtô sử dụng nhiên liệu xăng.
- Xe ôtô sử dụng nhiên liệu dầu diezel.
- Xe ôtô sử dụng nhiên liệu khí gas hoặc gas hoá lỏng.
- Xe ô tô sử dụng nhiên liệu xăng kết hợp sử dụng năng lượng điện.
- Xe ôtô sử dụng năng lượng điện.

1.2.3 - Theo công dụng
Theo công dụng ôtô được chia thành các loại:
- Ôtô chở hàng, bao gồm: Ôtô tải, ôtô tải tự đổ, ôtô tải có cần cẩu...
- Ôtô chở người, bao gồm: Ôtô buýt, ôtô ...

===== KẾT QUẢ 2 =====
Source: None
Page: None
Nội dung:
Để làm mát động cơ, hiện nay thường sử dụng:
- Hệ thống làm mát bằng không khí.
- Hệ thống làm mát bằng nước.

Hệ thống làm mát bằng không khí thường được sử dụng trên các loại ôtô chạy ở những vùng sa mạc hoặc ở những nơi thiếu nước. Hệ thống làm mát bằng nước có nhiều ưu điểm nên được sử dụng rộng rãi trên các loại động cơ ôtô. ...

===== KẾT QUẢ 3 =====
Sourc